<a href="https://colab.research.google.com/github/AryanWadhwa05/Stock-Analysis-and-Prediction/blob/main/streamlittestdeploy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Run this cell to load some data
!pip install streamlit
!pip install pyngrok 
!pip install yfinance

import tensorflow as tf
import pandas as pd
import cv2
import plotly.express as px
import numpy as np
import yfinance as yf
import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import plotly.express as px
import re
import numpy as np
import datetime as dt
%matplotlib inline
from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler

# %%writefile app.py

In [ ]:
%%writefile app.py

import streamlit as st
import yfinance as yf
import pandas as pd
import matplotlib.pyplot as plt
from pandas_datareader import data as pdr
import plotly.express as px
import re
import numpy as np
import datetime as dt

from matplotlib.pylab import rcParams
rcParams['figure.figsize']=20,10
from keras.models import Sequential
from keras.layers import LSTM,Dropout,Dense
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# download dataframe
st.title("Stock Market Analysis")

#sidebar
st.sidebar.title("Stocks")

st.sidebar.subheader("Stock Search")

stock_search_test = st.sidebar.text_input('Stock Search')
display_cols = st.sidebar.multiselect('Display Data', ["Close","Adj Close","Low","Open","High","Volume"],default = "High")
graph_type = st.sidebar.selectbox('Graph Type', ["Line","Bar","Vega Lite","AltAir","High","Volume"])

start_date = st.sidebar.date_input('Start Date')
end_date = st.sidebar.date_input('End Date')

yf.pdr_override()
tickers = ""

tickers = stock_search_test.replace(","," ")

data = yf.download(tickers, start=start_date, end=end_date)

dataframe = data[display_cols]
dataframe.reset_index(level=0, inplace=True)

# dataframe.replace('^\b+', '', regex=True, inplace=True)
st.write(dataframe)

plt.plot(dataframe["Date"],dataframe["High"])
# plt.legend(dataframe["High"])
plt.ylabel("Dollars")
plt.xlabel("Date")
plt.draw()

st.pyplot(plt)

# # machine learning starts
cf=dataframe

# cf["Date"]=pd.to_datetime(cf.Date,format="%Y-%m-%d")
cf.index=cf['Date']

# st.write(dataframe)

# plt.figure(figsize=(16,8))
# plt.plot(cf["Close"],label='Close Price history')

data=cf.sort_index(ascending=True,axis=0)
new_dataset=pd.DataFrame(index=range(0,len(cf)),columns=['Date',"High"])

for i in range(0,len(data)):
    new_dataset["Date"][i]=data['Date'][i]
    new_dataset[display_cols[0]][i]=data[display_cols[0]][i]

new_dataset["Date"].apply('str')
final_dataset=new_dataset.values

train_data=[]
valid_data=[]

#splitting the dataset
train_data_number = int((len(final_dataset)*0.8))
train_data=final_dataset[0:train_data_number,:]
valid_data=final_dataset[train_data_number:,:]

new_dataset.index=new_dataset.Date
new_dataset.drop("Date",axis=1,inplace=True)

#Feature Scaling
scaler=MinMaxScaler(feature_range=(0,1))
scaled_data=scaler.fit_transform(new_dataset)

# new_dataset.reset_index(level=0, inplace=True)
# new_dataset["High"] = scaled_data

# st.write(new_dataset)

x_train_data,y_train_data=[],[]

for i in range(60,len(train_data)):
    x_train_data.append(scaled_data[i-60:i,0])
    y_train_data.append(scaled_data[i,0])
    
x_train_data,y_train_data=np.array(x_train_data),np.array(y_train_data) #(len(data),60))

x_train_data=np.reshape(x_train_data,(x_train_data.shape[0],x_train_data.shape[1],1)) #(len(data), 60, 1)

# lstm architecture
lstm_model=Sequential()
lstm_model.add(LSTM(units=50,return_sequences=True,input_shape=(x_train_data.shape[1],1)))
lstm_model.add(LSTM(units=50))
lstm_model.add(Dense(1))

inputs_data=new_dataset[len(new_dataset)-len(valid_data)-60:].values
inputs_data=inputs_data.reshape(-1,1)
inputs_data=scaler.transform(inputs_data)

lstm_model.compile(loss='mean_squared_error',optimizer='adam')
lstm_model.fit(x_train_data,y_train_data,epochs=25,batch_size=20,verbose=2)

#Testing
X_test=[]
for i in range(60,inputs_data.shape[0]):
    X_test.append(inputs_data[i-60:i,0])
X_test=np.array(X_test)

X_test=np.reshape(X_test,(X_test.shape[0],X_test.shape[1],1))
predicted_closing_price=lstm_model.predict(X_test)
predicted_closing_price=scaler.inverse_transform(predicted_closing_price)

train_data=new_dataset[:train_data_number]
valid_data=new_dataset[train_data_number:]

valid_data['Predictions']=predicted_closing_price
plt.plot(train_data["High"])
plt.plot(valid_data[['High',"Predictions"]])
st.pyplot(plt)

st.write(valid_data)


Writing app.py


In [ ]:
from pyngrok import ngrok
#Publish Web App (Run this again whenever you make changes)
public_url = ngrok.connect(port='80')
print (public_url)
!streamlit run --server.port 80 app.py

2021-10-25 02:27:23.258 INFO    pyngrok.ngrok: Opening tunnel named: http-80-0651c474-47b0-4fa4-a5df-3267be3658a0


2021-10-25 02:27:28.717 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:28+0000 lvl=info msg="no configuration paths supplied"
2021-10-25 02:27:28.719 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:28+0000 lvl=info msg="using configuration at default config path" path=/root/.ngrok2/ngrok.yml
2021-10-25 02:27:28.721 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:28+0000 lvl=info msg="open config file" path=/root/.ngrok2/ngrok.yml err=nil
2021-10-25 02:27:28.722 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:28+0000 lvl=info msg="starting web service" obj=web addr=127.0.0.1:4040
2021-10-25 02:27:28.946 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:28+0000 lvl=info msg="tunnel session started" obj=tunnels.session
2021-10-25 02:27:28.947 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:28+0000 lvl=info msg="client session established" obj=csess id=4c36c91bffe3
2021-10-25 02:27:28.955 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:28+0000 lvl=info msg=start pg=/api/tunnel

NgrokTunnel: "http://ffeb-34-125-79-76.ngrok.io" -> "http://localhost:80"

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.2:80
  External URL: http://34.125.79.76:80

[*********************100%***********************]  0 of 0 completed
2021-10-25 02:27:33.311 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 126, in download
    keys=shared._DFS.keys())
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/concat.py", line 284, in concat
    sort=sort,
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/concat.py", line 331, in __init__
    raise ValueError("No objects to concatenate")
ValueError: No objects to concatenate

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 354, in _run_script
    exec(code, module.__dict__)
  File

2021-10-25 02:27:35.715 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:35+0000 lvl=info msg="join connections" obj=join id=18b2f94fcb0a l=127.0.0.1:80 r=122.177.74.239:27486
2021-10-25 02:27:36.189 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:36+0000 lvl=info msg="join connections" obj=join id=d3993b1f610f l=127.0.0.1:80 r=122.177.74.239:32081
2021-10-25 02:27:40.885 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:40+0000 lvl=info msg="join connections" obj=join id=2ed3f810dc74 l=127.0.0.1:80 r=122.177.74.239:7222
2021-10-25 02:27:41.791 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:41+0000 lvl=info msg="join connections" obj=join id=6ef299f73893 l=127.0.0.1:80 r=122.177.74.239:27486
2021-10-25 02:27:41.826 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:41+0000 lvl=info msg="join connections" obj=join id=3288948723dc l=127.0.0.1:80 r=122.177.74.239:19576
2021-10-25 02:27:42.124 INFO    pyngrok.process.ngrok: t=2021-10-25T02:27:42+0000 lvl=info msg="join connections" ob

[*********************100%***********************]  0 of 0 completed
2021-10-25 02:39:21.326 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 126, in download
    keys=shared._DFS.keys())
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/concat.py", line 284, in concat
    sort=sort,
  File "/usr/local/lib/python3.7/dist-packages/pandas/core/reshape/concat.py", line 331, in __init__
    raise ValueError("No objects to concatenate")
ValueError: No objects to concatenate

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 354, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 39, in <module>
    data = yf.download(tickers, start=start_date, end=end_date)
  File "/usr/local/lib/python3.7/dist-packages/yfinance/multi.py", line 130, in download
    keys=shar

...

2021-10-25 02:39:29.201 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 354, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 89, in <module>
    scaled_data=scaler.fit_transform(new_dataset)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/base.py", line 571, in fit_transform
    return self.fit(X, **fit_params).transform(X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py", line 339, in fit
    return self.partial_fit(X, y)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py", line 373, in partial_fit
    force_all_finite="allow-nan")
  File "/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py", line 586, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape=(0, 1)) while a minimum of 1 is required by MinMaxScaler.



2021-10-25 02:39:29.676 INFO    pyngrok.process.ngrok: t=2021-10-25T02:39:29+0000 lvl=info msg="join connections" obj=join id=468e1e37e6bd l=127.0.0.1:80 r=122.177.74.239:4834


2021-10-25 02:39:41.551 NumExpr defaulting to 2 threads.
[*********************100%***********************]  1 of 1 completed
2021-10-25 02:39:42.888 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 354, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 97, in <module>
    x_train_data.append(scaled_data[i-60:i,0])
NameError: name 'x_train_data' is not defined

[*********************100%***********************]  1 of 1 completed
2021-10-25 02:41:05.275222: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2021-10-25 02:41:05.275340: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (8a1f5a1b0687): /proc/driver/nvidia/version does not exist
2021-10-25 02:41:05.981400: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLI

2021-10-25 02:46:56.535 INFO    pyngrok.process.ngrok: t=2021-10-25T02:46:56+0000 lvl=info msg="join connections" obj=join id=031aa904734f l=127.0.0.1:80 r=122.177.74.239:25684


[*********************100%***********************]  1 of 1 completed

1 Failed download:
- APPLE: No data found, symbol may be delisted
2021-10-25 02:47:48.158 Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/streamlit/script_runner.py", line 354, in _run_script
    exec(code, module.__dict__)
  File "/content/app.py", line 89, in <module>
    scaled_data=scaler.fit_transform(new_dataset)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/base.py", line 571, in fit_transform
    return self.fit(X, **fit_params).transform(X)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py", line 339, in fit
    return self.partial_fit(X, y)
  File "/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_data.py", line 373, in partial_fit
    force_all_finite="allow-nan")
  File "/usr/local/lib/python3.7/dist-packages/sklearn/utils/validation.py", line 586, in check_array
    context))
ValueError: Found array with 0 sample(s) (shape